In [1]:
import os, pandas as pd
from dotenv import load_dotenv
from utility import Utils as U
import sqlalchemy

#### Pull tables to staging schema

In [2]:
load_dotenv()
conn = U.create_db_connection(os.getenv('username'), os.getenv('password'),os.getenv('db_host'),os.getenv('db_name') )

In [ ]:
# same as before is there a shorter less repetitive way of pulling all tables?

In [3]:
Countries_df = U.fetch_db_table('Countries', conn, 'Staging')
Themes_df = U.fetch_db_table('Themes', conn, 'Staging')
Regions_df = U.fetch_db_table('Regions', conn, 'Staging')
Recommendations_df = U.fetch_db_table('Recommendations', conn, 'Staging')
tags_df = U.fetch_db_table('Tags', conn, 'Staging')
tags_matrix = U.fetch_db_table('TagsMatrix', conn, 'Staging')

     index  Country id                        Country name  \
0        0           1                              Uganda   
1        1           2  Venezuela (Bolivarian Republic of)   
2        2           3                             Hungary   
3        3           4                             Ireland   
4        4           5                            Thailand   
..     ...         ...                                 ...   
144    144         145                                Togo   
145    145         146                          Montenegro   
146    146         147                                Chad   
147    147         148                                Oman   
148    148         149                             Algeria   

                             Regions  
0                             Africa  
1    Latin America and the Caribbean  
2                    Eastern Europe   
3            Western Europe & Others  
4                       Asia-Pacific  
..                   

In [ ]:
Countries_df = Countries_df.merge(Regions_df, left_on="Regions", right_on="Regions")
Countries_df = Countries_df.drop(columns=["Regions"])

In [ ]:
Country_to_recommendations = df3[['Countries Concerned','UPR Recommending States']]
Country_to_recommendations.index = Country_to_recommendations.index.set_names(['Recommendation_id'])
Country_to_recommendations = Country_to_recommendations.reset_index(level=0).drop_duplicates(subset='Recommendation_id', keep='first')
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='Countries Concerned', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['Countries Concerned','Country name']).rename(columns={"Country id": "Countries Concerned"})
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='UPR Recommending States', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['UPR Recommending States','Country name']).rename(columns={"Country id": "UPR Recommending States"})

In [ ]:
Recommendation_to_theme = UPR_cycle_cleaned[['Themes']]
Recommendation_to_theme.index = Recommendation_to_theme.index.set_names(['Recommendation id'])
Recommendation_to_theme = Recommendation_to_theme.reset_index(level=0)

In [ ]:
Recommendation_to_theme = Recommendation_to_theme.merge(Themes_df, left_on='Themes', right_on='Themes')
Recommendation_to_theme = Recommendation_to_theme.drop(columns=['Themes']).rename(columns={'Theme id':'Themes'})

In [ ]:
tags_to_themes = pd.melt(tags_matrix, value_vars=tags_matrix.columns.tolist(), var_name='Tag', value_name='Theme')
tags_to_themes = tags_to_themes.merge(tags_df[["Tags","Tag id"]], left_on='Tag', right_on='Tags').drop(columns=['Tags', 'Tag'])
tags_to_themes = tags_to_themes.merge(Themes_df[["Themes","Theme id"]], left_on='Theme', right_on='Themes').drop(columns=['Themes', 'Theme'])

### Normalized join table Dataframes

In [ ]:
Country_to_recommendations

In [ ]:
Recommendation_to_theme

In [ ]:
tags_to_themes

#### Upload tables to Production schema